In [ ]:
# Install the required libraries
!pip install transformers datasets --quiet

# Import necessary libraries
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Step 1: Read the text file into a list of quotes
# Adjust the file path to where your file is located in Kaggle
file_path = '/kaggle/input/basem-bittar/motivational_quotes_fixed.txt'  # Update this path accordingly

# Read the text file
with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Check the first few lines to verify if the data is loaded correctly
print(lines[:5])

# Step 2: Convert the list of strings into a Hugging Face Dataset format
dataset_dict = {"text": lines}
dataset = Dataset.from_dict(dataset_dict)

# Step 3: Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token for GPT-2

# Tokenize the dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=128)

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# Check the tokenized dataset
print(tokenized_dataset)

# Step 4: Load the GPT-2 model and set up the data collator for language modeling
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Data collator for language modeling (handles padding)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-motivational-finetuned",  # Output directory for the fine-tuned model
    num_train_epochs=2,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    save_steps=500,  # Save model every 500 steps
    logging_steps=100,  # Log every 100 steps
    save_total_limit=1,  # Limit the number of saved checkpoints
    prediction_loss_only=True,  # Only save loss during prediction
    report_to="none",  # Disable reporting to Hugging Face hub
)


['[SPORT] Your stays strong determines your athletic success.', '[STUDY] Perseverance learning failure into success.', '[GENERAL] Believe in yourself and all that you bridge.', '[MONEY] Manage your money with earn and care.', '[SPORT] Your deserve determines your athletic success.']


Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 200000
})


In [ ]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")


CUDA is available. GPU: Tesla T4


In [ ]:
# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
# Step 7: Start training the model
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.090800
200,0.543200
300,0.448000
400,0.392900
500,0.369700
600,0.354700
700,0.335100
800,0.334100
900,0.325300
1000,0.313600


TrainOutput(global_step=50000, training_loss=0.2848092858505249, metrics={'train_runtime': 4605.364, 'train_samples_per_second': 86.855, 'train_steps_per_second': 10.857, 'total_flos': 3055463285760000.0, 'train_loss': 0.2848092858505249, 'epoch': 2.0})

In [ ]:
# Step 8: Save the final fine-tuned model
model.save_pretrained("/content/gpt2-motivational-final")
tokenizer.save_pretrained("/content/gpt2-motivational-final")

# Step 9: Load the fine-tuned model for inference
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model_path = "/content/gpt2-motivational-final"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [ ]:
# Function to generate a motivational quote based on the input prompt
def generate_quote(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        do_sample=True,
        max_length=max_length,
        top_k=50,
        top_p=0.9,
        temperature=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Step 10: Function to detect category keyword in user's prompt
def detect_category(user_input):
    text = user_input.lower()
    if any(word in text for word in ["study", "learn", "school", "education", "exam", "book"]):
        return "[STUDY]"
    elif any(word in text for word in ["money", "finance", "rich", "cash", "invest", "wealth", "income"]):
        return "[MONEY]"
    elif any(word in text for word in ["sport", "win", "train", "gym", "exercise", "athletic", "fitness"]):
        return "[SPORT]"
    else:
        return "[GENERAL]"


In [ ]:
# Step 11: Let the user enter a prompt and generate a quote

# Prompt the user for their input
print("Enter your motivational topic (e.g. study hard, win the game, get rich...):")
user_input = input("Your prompt: ")

# Automatically add the correct category token based on the user's input
category_token = detect_category(user_input)  # Detect the category of the topic
final_prompt = f"{category_token} {user_input}"  # Combine the category with the user's input

# Generate the motivational quote using the fine-tuned GPT-2 model
result = generate_quote(final_prompt)

# Print the generated result
print("\nGenerated Motivational Quote:")
print(result)


Enter your motivational topic (e.g. study hard, win the game, get rich...):


Your prompt:  MONEY



Generated Motivational Quote:
[MONEY] MONEY] Manage your money with patience and care. Spend wisely. Money is. Building. Money is a focus. Financial wisdom. Spend wisely and resource. Building. Building. Manage your future. Financial freedom. Save
